<a href="https://colab.research.google.com/github/jdldeauna/GitHub-Issues/blob/master/RMACC_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamental Machine Learning Pipeline applied to Atmospheric Science Data

In this tutorial, we will explore different aspects of the machine learning (ML) pipeline using weather data. This includes pre-processing techniques, how to split data for training\validation\testing the models, how to train basic ML models, and finally how to evaluate the model performance. 

I will go through lecture-like materials, with links to (hopefully) helpul resources, and coding exercises. 

Please cite the notebook as follows:

    Burke, A., 2021: "Fundamental Machine Learning Pipeline applied to Atmospheric Science Data"

The data for this tutorial can be cited as: 

    McGovern, A., Burke, A., Harrison, D., and G. M. Lackmann, 2020: A Machine Learning Tutorial for Operational Forecasting: Part I. Wea. Forecasting, In Press 


<br> 
<br> 
**My email: aburke1@ou.edu**




<br>


# Setup


<br>

In [ ]:
import pandas as pd
import numpy as np
import gdown
! pip install netcdf4
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression

import tensorflow as tf
import keras.models as models
import keras.layers as layers

import warnings
warnings.filterwarnings('ignore')


### Prevent Auto-scrolling

The next cell prevents output in the notebook from being nested in a scroll box

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;

## Import data

The next cell imports all of the data that will be used by this notebook. If anything crashes, it will probably be here.

In [ ]:
! wget https://zenodo.org/record/4773839/files/AI_tutorial_data.tar.gz -O AI_tutorial_data.tar.gz
! tar -xzvf AI_tutorial_data.tar.gz
! rm *.tar.gz 

<br>


# Tutorial Overview


<br>


**Brief:** 
1. General ML Pipeline
2. Prediction Problem and Dataset Information

**More in Depth:**
3. Data Pre-processing
4. Model Training
  - Linear Regression
  - Artificial Neural Network
5. Model evaluation
6. Exercise

[Here](https://towardsdatascience.com/list-of-free-must-read-machine-learning-books-89576749d2ff) is a list of free books about ML from a statistical perspective.
 


<br>


# **1. General ML pipeline**




### The (brief) end-to-end process for using ML models

*Define the Problem*
  - Look at the data and forecasting task to see if ML is appropriate. Do existing methods produce skillful results? If so then ML may not be necessary.
  - Is the data normally distributed, correlated, etc.? **Do you have enough data if what you are trying to predict is rare?**

*Separate Data* 
  - Split your data into **at least two independent sets**: training and testing
  - Possibly use validation set to tune ML models before applying them to the testing set. 

*Data Pre-processing* 
  - After determining ML would benefit your problem domain, transform the data to be more amenable for ML
  - Find out what shape the data needs to be (vectors for ML versus tensors for DL)
  - Normalize/Scale data
  
### ***The above steps are at least 80% of the work when working with ML models***


*Model Training*
  - Optimize ML model(s) using the **training dataset only**

*Model Deployment*
  - Apply the trained ML model to the **testing dataset only** 

*Model Evaluation*
  - Use verification metrics and subjective evaluations to determine the skill of the ML model for a given predictive task.  

*Model Interpretation*
  - Use different interpretation techniques (permutation variable importance, partial dependence plots, backwards optimization, etc.) to determine **why** the ML model(s) make decisions
  - Evaluate if the decision-making process is reasonable
  - Allow end-users to better trust the ML decisions

<br>
<br>

## These steps are applicable for both ML and DL models. More details about the ML pipeline can be found [here](https://www.oreilly.com/library/view/building-machine-learning/9781492053187/ch01.html). 

<br>
<br>


<br>


# **2. Prediction Problem and Dataset Information**



<br>




<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/NWS_Weather_Forecast_Offices.svg/720px-NWS_Weather_Forecast_Offices.svg.png" width="600">


We are going to mimic the [WxChallenge](https://www.wxchallenge.com/) (with slight changes for time constraints)
  - Predict the high/low temperature and wind speed of a given city
  - Relatively straight forward problem that is familiar to many meteorologists 
  - Learn the ML process on one city and apply your knowledge to another city or an entirely different prediction problem, it is up to you! (See section 9) 


 **Dataset Information**
- Numerical Weather Prediction (NWP) point forecasts from the Iowa Environmental Mesonet between 1 January 2011 to 1 August 2019
- 24-hour forecast runs from the GFS, NAM, RAP, and NAM 4km at 0000, 0600, 1200, and 1800 UTC
- 70 hourly forecast variables for each model 
- NMP variables processed into 24-hour maximum, minimum, and average values for the 0600 to 0600 UTC period (same as WxChallenge)
- If one NWP forecast is missing, replaced with average value from all other NWP forecasts for the given variable
- Observations are NWS ASOS point data for a given city  
- Days with missing observations were removed




<br> 


Load in the CSV data. *We will look at kdfw*

<br>

In [ ]:
total_dataset = pd.read_csv('AI_tutorial_data/kdfw_processed_data.csv',index_col=0).sort_values(by='date')
total_dataset = total_dataset.replace('********', np.nan).replace(np.inf,np.nan).dropna(how='any',axis=1)

In [ ]:
#First five rows
total_dataset.head()

In [ ]:
#Last five rows
total_dataset.tail()

In [ ]:
print('The columns (predictor/input variables) we have to work with are:\n')
total_dataset.columns

#Explore the different variables if you wish

# **3. Data Pre-processing**

## Partitioning Data

What would happen if we use all of our data to build a model and evaluate its accuracy?

***Let's try!*** (Do not need to know how to do this yet)

In [ ]:
#Observation data
total_label_data = total_dataset.filter(like='OBS')
print(total_label_data)

In [ ]:
#Variables used for training 
dropCols = list(total_label_data.columns) + ['date']
total_feature_data = total_dataset.copy(deep=True)
total_feature_data = total_feature_data.drop(dropCols,axis=1)
print(total_feature_data)

In [ ]:
max_temperature_obs = total_label_data['OBS_tmpf_max'][:2000]
first_50_features = total_feature_data.iloc[:2000,:50]

# # Train and evaluate the model to predict max temperature
linear_regression_model = LinearRegression()

#Train
linear_regression_model.fit(first_50_features.values,max_temperature_obs.values)
# Evaluate R^2
print(f'Testing data shape: {first_50_features.shape}')
print(f'Observation data shape: {max_temperature_obs.shape}')
score = linear_regression_model.score(first_50_features.values,max_temperature_obs.values)
print(f'R squared score: {score}')


What happens if we test on other data?

In [ ]:
max_temperature_obs = total_label_data['OBS_tmpf_max'][-500:]
first_50_features = total_feature_data.iloc[-500:,:50]

score = linear_regression_model.score(first_50_features.values,max_temperature_obs.values)
print(f'Testing data shape: {first_50_features.shape}')
print(f'Observation data shape: {max_temperature_obs.shape}')
print(f'R squared score: {score}')


### Dataset independence 
- **Critical** when deploying machine learning models. 
  - Training and evaluating on the same data gives an unrealistic view of how the model will perform on new data
  - Instead, split the data into **training** and **testing** datasets that are independent enough to provide a clear picture

### How much data should go in each subset?
  - Depends (I know, not the most ideal answer)
  - Typical train/test split is 80% for training and 20% for testing
    - Varies for the size of your dataset
    - Possibly need to use data augmentation to increase training data size 
    - Find more information on data augmentation for deep learning [here](https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/)

### **Lets split the data officially into training and testing datasets** 


In [ ]:
# Function to split data based on given dates
def split_data_year(input_data,labels,start_date_str,end_date_str):
  data = input_data.copy()
  date_list = pd.to_datetime(data['date'])
  date_mask = (date_list > start_date_str) & (date_list <= end_date_str)
  out_data = data.loc[date_mask,:].drop(['date'],axis=1)
  out_labels = labels.loc[date_mask,:]
  return out_data,out_labels

In [ ]:
# Remove MOS data based on previous work 
mosCols = [key for key in total_dataset.columns if 'MOS' in key]
total_dataset = total_dataset.drop(mosCols, axis = 1)
#Get total data
total_label_data = total_dataset.filter(like='OBS')
dropCols = list(total_label_data.columns)
total_feature_data = total_dataset.copy(deep=True).drop(dropCols,axis=1)

#Training data between 2011 and 2017
train_features, train_labels = split_data_year(total_feature_data,
    total_label_data,'2011-01-01','2017-12-31')

#Testing data between 2018 and 2019
#Keep a month between train/test data for greater likelihood 
# of dataset independence
test_features, test_labels = split_data_year(total_feature_data,
    total_label_data,'2018-02-01','2019-12-31')

## Transforming Data

### Normalization and Scaling

 
Why should you scale your data?
- Data variables with different scales (e.g. temperature versus sea-level pressure) affects what variables the ML models find important 
- Temperatures varying from $\sim$180-330 K will be prioritized over specific humidity values varying from $\sim$0-0.02 kg kg$^{-1}$

Normalizing or scaling data removes this issue.

**Important: We normalize/scale our data based on the training data and save these values to apply to testing data**

More details about each transformation method can be found [here](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)


***MinMaxScaler***

Scalar = $\frac{x_{i} - min(x)}{max(x) - min(x)}$

Scales data between 0 and 1 based on training data minimum and maximum values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Fit/find minimum and maximum value for each input variable
min_max_model = MinMaxScaler().fit(train_features)

# This saves the values to apply or 'transform' testing data
min_max_train_features = min_max_model.transform(train_features)

print('Training data range \nMax:',np.nanmax(min_max_train_features),
    ', Min:',np.nanmin(min_max_train_features))


#################################
# Apply the min/max values to testing data
min_max_test_features = min_max_model.transform(test_features)

print('\nTesting data range \nMax:',np.nanmax(min_max_test_features),
    ', Min:',np.nanmin(min_max_test_features))

### **The scaled data for the rest of the tutorial**

**My preferred scaling method**

- Replace any outlier values ( > 3 standard deviations from mean) with the training data mean

- Use MinMaxScaler after replacement

- Reduces the chance outliers affect training scaling values

In [ ]:
no_outlier_train_features = train_features.copy()
no_outlier_test_features = test_features.copy()

In [ ]:
train_standard_dev = train_features.std()
train_mean = train_features.mean()

for column in train_features.columns:
  outlier_threshold_value = 3.0*train_standard_dev[column]
  
  #pandas.where() documentation:
  #Where cond is True, keep the original value. Where False, replace with corresponding value from other
  no_outlier_train_features[column].where(
      np.abs(train_features[column]-train_mean[column]) < outlier_threshold_value, 
      train_mean[column],
      inplace=True)
  
  no_outlier_test_features[column].where(
      np.abs(test_features[column]-train_mean[column]) < outlier_threshold_value, 
      train_mean[column],
      inplace=True)
  
from sklearn.preprocessing import MinMaxScaler

no_outlier_min_max_model = MinMaxScaler().fit(no_outlier_train_features)
scaled_no_outlier_train_features = no_outlier_min_max_model.transform(no_outlier_train_features)
scaled_no_outlier_test_features = no_outlier_min_max_model.transform(no_outlier_test_features)

If you only want the data for the rest of the tutorial, do not run the cell below (it will return an error).

If you're going through the entire tutorial up until now, go ahead and run the cell. 

In [ ]:
print('Previous MinMaxScaler\n')
print(
    f'Training data range \nMax: {np.nanmax(min_max_train_features)}, '+
    f'Min: {np.nanmin(min_max_train_features)}')
print(
    f'Testing data range \nMax: {np.nanmax(min_max_test_features)}, '+
    f'Min: {np.nanmin(min_max_test_features)}')

print('\n\nNo outlier MinMaxScaler\n')
print(
    f'Training data range \nMax: {np.nanmax(scaled_no_outlier_train_features)}, '+
    f'Min: {np.nanmin(scaled_no_outlier_train_features)}')
print(
    f'Testing data range \nMax: {np.nanmax(scaled_no_outlier_test_features)}, '+
    f'Min: {np.nanmin(scaled_no_outlier_test_features)}')

### Dimensionality Reduction


**Why perform dimensionality reduction?**
- Requires less space to store data
- Model training is faster on smaller dimensional datasets
- Only train on relevant features (reduces redundancy)
- Easier to visualize in lower dimensional space

<br>

**Replacing outliers with mean training data values (previous scaling step) is crucial when using dimensionality reduction with the WxChallenge data**

...We had to learn this the hard way!


#### Principal Component Analysis (PCA) 


PCA reduces the dimensionality of a dataset by creating new variables that are linear combinations of the original features. This way, less variables are needed to encompass the variability of the original data and redundant information is reduced. 

<br> 

The *principal components (PCs)* or new variables are linearly independent and uncorrelated, which can be beneficial for certain model assumptions. The first PC is a new variable with the most variance from the original dataset, the second PC is a new variable with the second most variance, etc. 


<br> 


Scaling is really important so that the PCs do not emphasize variables simply because they have large data ranges.

Find more information [here](https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c
) and [here](
https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60).


<img src="https://miro.medium.com/max/1280/1*mYuvXAmJmkqYuEoGsYCRwQ.gif" width="500">

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
print(f'Before PCA: \n{scaled_no_outlier_train_features[0,:10]}')

#Fit the PCA model with the already scaled data
explorative_pca_model = PCA().fit(scaled_no_outlier_train_features)

#Plot the cumulutive sum of the explained variance ratio 
plt.plot(np.cumsum(explorative_pca_model.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained variance')
plt.grid(True)
#If more than two components are wanted, fit the model again and then transform

#Otherwise transform the desired data using a fitted PCA model
exporative_pca_train_features = explorative_pca_model.transform(scaled_no_outlier_train_features)

print('After PCA: \n' + str(exporative_pca_train_features[0,:10]))

Actual PCA model based on previous work, **experiment with 'n_components' value for different number of input features**

In [ ]:
pca_model = PCA(n_components = 0.65, svd_solver = 'full').fit(scaled_no_outlier_train_features)
pca_train_features = pca_model.transform(scaled_no_outlier_train_features)
pca_test_features = pca_model.transform(scaled_no_outlier_test_features)

In [ ]:
pca_train_features.shape

# **4. Model Training**

## Linear Regression Model

Linear regression fits the following equation to the training data.

$\hat{y} = \beta_0 + \sum\limits_{j = 1}^{M} \beta_j x_j$

$x$: Predictor variable

$M$: Number of predictor variables

$\beta$: Coefficient for predictor variable

$\beta_0$: Bias coefficient/intercept

$\hat{y}$: Predicted label value

<br>
<br>

The coefficient terms, $\beta$ and $\beta_0$ are adjusted during training to minimize the mean square error (MSE) between the predicted  label $\hat{y}$ and true label $y$. 

$\textrm{MSE} = \frac{1}{N} \sum\limits_{i = 1}^{N} (\hat{y}_i - y_i)^2$

$N$: Number of training labels

<br> 

Combining the two equations using the $\hat{y}$ term yields:

$\textrm{MSE} = \frac{1}{N} \sum\limits_{i = 1}^{N} (\beta_0 + \sum\limits_{j = 1}^{M} \beta_j x_{ij} - y_i)^2$

**We want are $\beta$ and $\beta_0$ values that result in predicted values $\hat{y}$  closest to the true label  ${y}$.**


<br>

To do this, we take the derivative of each coefficient with respect to the MSE equation and minimize this value. This is why linear regression is sometimes referred to **least-squares linear regression**

The derivatives of model coefficients with respect to MSE are as follows.

$\frac{\partial}{\partial \beta_0}(\textrm{MSE}) = \frac{2}{N} \sum\limits_{i = 1}^{N} (\hat{y}_i - y_i)$
$\frac{\partial}{\partial \beta_j}(\textrm{MSE}) = \frac{2}{N} \sum\limits_{i = 1}^{N} x_{ij} (\hat{y}_i - y_i)$

The coefficients are updated throughout training by: 

$\beta_0 \leftarrow \beta_0 - \alpha \frac{\partial}{\partial \beta_0}(\textrm{MSE})$

$\beta_j \leftarrow \beta_j - \alpha \frac{\partial}{\partial \beta_j}(\textrm{MSE})$


$\alpha$: Gradient descent parameter (how much do you want to change the coefficients at each update iteration)





<img src="https://bs-uploads.toptal.io/blackfish-uploads/uploaded_file/file/238279/image-1587548990037-a9dc244b868173eb48df437eedc72910.gif" width="600">





## Artificial Neural Network

The rest of the methods do not delve deeply into the inputs or outputs of a DL network. Instead they example the ***hidden layers*** and how each hidden neuron ***fires*** or ***activates***. 

<br>

Neural networks are based on the processes behind neurons in the brain. If a certain neuron ***fires*** then information is passed along it. Otherwise  the information stops there. 


<img src="https://i.pinimg.com/originals/1c/63/b5/1c63b58aa68d9fee506e2397d05598e2.gif" width="800">

[Image credit](https://www.dailymail.co.uk/sciencetech/article-2581184/The-dynamic-mind-Stunning-3D-glass-brain-shows-neurons-firing-real-time.html)


<br>

Similarly, only a hidden layer neuron that ***fires*** or ***activates*** passes along information.

**NOTE:  There are a lot of good visualizations at https://distill.pub that I recommend looking at. I personally find them not only informative but fun as well.**

Artificial Neural Network (ANN)​

- Every data point associated with a label, no learning between points​

- Removes any dependences in space or time​

- Using a number of 'hidden layers' the network predicts based on the given data and then adjusts itself based on how wrong its prediction is to the truth​

- Basically, a bunch of linear regressions sandwiched with non-linear *sparkle* ​

<img src="https://miro.medium.com/max/1200/1*36MELEhgZsPFuzlZvObnxA.gif" width="800">


​[Image credit](https://miro.medium.com/max/1200/1*36MELEhgZsPFuzlZvObnxA.gif)

<br>

<img src="https://www.researchgate.net/profile/Facundo-Bre/publication/321259051/figure/fig1/AS:614329250496529@1523478915726/Artificial-neural-network-architecture-ANN-i-h-1-h-2-h-n-o.png" width="800">

​[Image credit](https://www.researchgate.net/profile/Facundo-Bre/publication/321259051/figure/fig1/AS:614329250496529@1523478915726/Artificial-neural-network-architecture-ANN-i-h-1-h-2-h-n-o.png)


 

In an artificial neural network, each neuron has inputs of weighted sums with a bias term added. 
- **Weights** are updated and *learned* based on how far off the networks prediction is from the true observation. 
- Same as gradient decent above. 

<img src="https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-09-at-3-42-21-am.png" width="500">

​[Image credit](https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-09-at-3-42-21-am.png)

<br> 

<img src="https://miro.medium.com/max/1200/1*ZafDv3VUm60Eh10OeJu1vw.png" width="600">


​[Image credit](https://miro.medium.com/max/1200/1*ZafDv3VUm60Eh10OeJu1vw.png)


## Hyperparameter Tuning

### **Validation**

  - What if you don't want to use the python models out-of-the-box, but instead change the **hyperparameters** to better fit your data?

  - Hyperparameters: user-defined parameters that constrain a model to speed up learning and/or prevent overfitting (McGovern et al. 2020) 


<img src="https://miro.medium.com/max/875/1*_7OPgojau8hkiPUiHoGK_w.png" width="600">


  - Two (main) possibilities: 
    - Create a third independent subset for tuning hyperparameters, called  **validation dataset** 
    - Use cross-validation on the already divided training data if dataset is small


### **Cross-Validation**
  - Break up the training dataset into multiple **folds**, where a certain number are used to **fit** or **train** the ML model and a single fold (not used in training) **validates** the model (essentially testing the model)
  - Repeat this procedure of breaking up training data into training/validation folds until every data fold is used for validation
  - The trained ML model with the highest validation score of all the different training/validation pairs is evaluated on the testing data
    - **Testing data is never used in training the model, only for evaluating a model already trained**


The easiest way to do this in python is with [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)


<img src="https://docs.google.com/uc?export=download&id=1Llru509goS8OA3irMXEmoCoDMDPVhUvS" height= "400">

### **Different Linear Regression Hyperparameters**


From the [sklearn Linear Regression documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html): 
 
```
 class sklearn.linear_model.LinearRegression(*, fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
 ```

Each of these can be changed when training the LinearRegression model.

<br>

No changes:

```
LinearRegression()
```

Allow the algorithm to calculate the intercept: 

```
LinearRegression(fit_intercept=True)
```

Normalize the data before regression (*Usually false so a user can decide how to scale data*):
```
LinearRegression(normalize=False)
```

Copy the input feature data for training, else the data can be overwritten

```
LinearRegression(copy_X=True)
```

Train the model with multiple jobs/processes if n_jobs > 1 

```
LinearRegression(n_jobs=None)
```







## Fitting Linear Regression Model

Using our scaled data without outliers, train the LinearRegression (LinReg) model out-of-the-box (no change in hyperparameters)

*We will have different LinReg models predicting the high temperature, low temperature, and wind speed.* 


In [ ]:
from sklearn.linear_model import LinearRegression
linreg_model = LinearRegression()
# High Temperature Model
high_temp_linreg = linreg_model.fit(pca_train_features,train_labels['OBS_tmpf_max'].values)

linreg_model = LinearRegression()
# Low Temperature Model
low_temp_linreg = linreg_model.fit(pca_train_features,train_labels['OBS_tmpf_min'].values)

wind_linreg_model = LinearRegression()
# Max Wind Speed Model
wind_linreg = wind_linreg_model.fit(pca_train_features,train_labels['OBS_sknt_max'].values)

## Fitting Neural Network 

In [ ]:

def ANN_WxChallenge(input_data,obs_data): 
  print( input_data.shape)
  input_shape = (input_data.shape[1],)
  print(input_shape)
  # # Artificial Neural network
  ann_model = models.Sequential()
  ann_model.add(layers.GaussianNoise(0.01, input_shape=input_shape))

  ann_model.add(layers.Dense(128))
  ann_model.add(layers.BatchNormalization())
  ann_model.add(layers.LeakyReLU(alpha=0.1))

  ann_model.add(layers.Dense(64))
  ann_model.add(layers.BatchNormalization())
  ann_model.add(layers.LeakyReLU(alpha=0.1))

  ann_model.add(layers.Dense(32))
  ann_model.add(layers.BatchNormalization())
  ann_model.add(layers.LeakyReLU(alpha=0.1))

  ann_model.add(layers.Dense(1,activation='linear'))
  print(ann_model.summary())

  ann_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
  history = ann_model.fit(input_data, obs_data, epochs=20, batch_size=64)

  return ann_model

In [ ]:
high_temp_ann =  ANN_WxChallenge(scaled_no_outlier_train_features[:,:50],
                                 train_labels['OBS_tmpf_max'].values)

low_temp_ann =  ANN_WxChallenge(scaled_no_outlier_train_features[:,:50],
                                 train_labels['OBS_tmpf_min'].values)

wind_ann =  ANN_WxChallenge(scaled_no_outlier_train_features[:,:50],
                                  train_labels['OBS_sknt_max'].values)

# **5. Model Evaluation**

Some common evaluation metrics used with regression problems are: 

Mean absolute error (MAE): $\frac{1}{N} \sum\limits_{i = 1}^{N} \lvert \hat{y}_i - y_i \rvert$

Mean squared error (MSE): $\frac{1}{N} \sum\limits_{i = 1}^{N} (\hat{y}_i - y_i)^2$

Mean signed error ("bias"): $\frac{1}{N} \sum\limits_{i = 1}^{N} (\hat{y}_i - y_i)$

MAE skill score: $\frac{\textrm{MAE}_{\textrm{climo}} - \textrm{MAE}}{\textrm{MAE}_{\textrm{climo}}}$

-  $\textrm{MAE}_{\textrm{climo}}$ is the average training MAE


More evalution metrics can be found [here](https://scikit-learn.org/stable/modules/model_evaluation.html). 




In [ ]:
from sklearn.metrics import mean_squared_error

high_temp_y_pred_linreg = high_temp_linreg.predict(pca_test_features)
high_temp_mse_linreg = mean_squared_error(high_temp_y_pred_linreg,test_labels['OBS_tmpf_max'].values)

print(f'Mean square error of Linear Regression: {high_temp_mse_linreg}')
print(f'\nPredicted values \n{high_temp_y_pred_linreg[:10]}')
print(f'\nActual values \n{test_labels["OBS_tmpf_max"].values[:10] }')

high_temp_y_pred_ann = high_temp_ann.predict(scaled_no_outlier_test_features[:,:50])
high_temp_mse_ann = mean_squared_error(high_temp_y_pred_ann,test_labels['OBS_tmpf_max'].values)
print(f'\n\nMean square error of ANN: {high_temp_mse_ann}')


Another type of verification/evaluation metric is the **reliability curve** 
- Shows model bias with respect to a conditional mean observation for each forecasted value
- What is the mean observed value for a given range of forecast values
- Data points above the 1-1 line are underforecasting, points under the line are overforecasting

In [ ]:
 def regression_reliability_curve(forecasts,observations,nbins=20):
  step = (np.nanmax(forecasts)-np.nanmin(forecasts))/nbins
  bins = np.arange(np.nanmin(forecasts),np.nanmax(forecasts),step)

  mean_forecast = np.empty( (np.shape(bins)) )*np.nan
  mean_obs = np.empty( (np.shape(bins)) )*np.nan

  # For each bin, find the mean forecast value and observation
  for b,bin in enumerate(bins[:-1]):
    bin_indices = np.where((forecasts >= bin) & (forecasts < bins[b+1]))[0]
    mean_forecast[b] = np.nanmean(forecasts[bin_indices])
    mean_obs[b] = np.nanmean(observations[bin_indices])

  #Get maximum value of both datasets to plot
  max_value = np.nanmax( [np.nanmax(forecasts) , np.nanmax(observations)] )
  perfect_values = np.arange(0.,max_value)

  plt.figure(figsize=(5,5))
  #Plotting 1-1 line
  plt.plot(perfect_values,perfect_values,linestyle='dashed',color='k')
  #Plotting mean forecast values versus the mean observations
  plt.plot(mean_forecast,mean_obs)
  plt.ylabel('Conditional Mean Observation',fontsize=16)
  plt.xlabel('Forecast value',fontsize=16)
  plt.title('Reliability Curve',fontsize=18)
  plt.show()
  return

In [ ]:
# Reliability curve of high temperature linear regression model
regression_reliability_curve(high_temp_y_pred_linreg, test_labels['OBS_tmpf_max'].values, nbins=20)

# **6. Exercise**

---

Select the city and data you would like to use. There are multiple WFO datapoints, to look at them run: 

```
! ls AI_tutorial_data
```

---



In [ ]:
! ls AI_tutorial_data


In [ ]:
data_file = 'AI_tutorial_data/kgeg_processed_data.csv'
total_dataset = pd.read_csv(data_file,index_col=0).sort_values(by='date')


###########################
# Do not need to change the following lines
###########################


# Remove MOS data, unknown strings, and NaN values
mosCols = [key for key in total_dataset.columns if 'MOS' in key]
total_dataset = total_dataset.replace('********', np.nan).replace(np.inf,np.nan).dropna(how='any',axis=1).drop(mosCols, axis = 1)

#Get label data
total_label_data = total_dataset.filter(like='OBS')

#Get feature data
dropCols = list(total_label_data.columns)
total_feature_data = total_dataset.copy(deep=True).drop(dropCols,axis=1)

---

Choose the train, validation, test split based on time periods

See **Data Pre-processing > Partioning Data** Section

---

In [ ]:
# Function to split data based on given dates
def split_data_year(input_data,labels,start_date_str,end_date_str):
  data = input_data.copy()
  date_list = pd.to_datetime(data['date'])
  date_mask = (date_list > start_date_str) & (date_list <= end_date_str)
  out_data = data.loc[date_mask,:].drop(['date'],axis=1)
  out_labels = labels.loc[date_mask,:]
  return out_data,out_labels

In [ ]:
###########################
# Will need to change the rest of the lines in the exercise
###########################


#Training data 
train_features, train_labels = split_data_year(total_feature_data,total_label_data,
      '2011-01-01','2017-12-31')

# #Validation data
# valid_features, valid_labels = split_data_year(total_feature_data,total_label_data,'','')

#Testing data
test_features, test_labels = split_data_year(total_feature_data,total_label_data,
                                             '2018-02-01','2019-12-31')

---
Apply transformations and Principal Component Analysis (optional) to data.

*I highly suggest using PCA so that the ML models train in a timely manner*


See  **Data Pre-processing > Normalization and Scaling** and **Data Pre-processing > Dimensionality Reduction > Principal Component Analysis** Sections. 


Other scaling tranformations [here](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py).

---

In [ ]:
# import PCA and/or metrics 


# First transform training data
no_outlier_train_features = train_features.copy()
no_outlier_test_features = test_features.copy()

train_standard_dev = train_features.std()
train_mean = train_features.mean()

for column in train_features.columns:
  outlier_threshold_value = 3.0*train_standard_dev[column]
  
  #pandas.where() documentation:
  #Where cond is True, keep the original value. Where False, replace with corresponding value from other
  no_outlier_train_features[column].where(
      np.abs(train_features[column]-train_mean[column]) < outlier_threshold_value, 
      train_mean[column],
      inplace=True)
  
  no_outlier_test_features[column].where(
      np.abs(test_features[column]-train_mean[column]) < outlier_threshold_value, 
      train_mean[column],
      inplace=True)
  
from sklearn.preprocessing import MinMaxScaler

no_outlier_min_max_model = MinMaxScaler().fit(no_outlier_train_features)
scaled_no_outlier_train_features = no_outlier_min_max_model.transform(no_outlier_train_features)

scaled_no_outlier_test_features = no_outlier_min_max_model.transform(no_outlier_test_features)

In [ ]:
pca = PCA(n_components=0.65,svd_solver='full')

pca.fit(scaled_no_outlier_train_features)
pca_train_features = pca.transform(scaled_no_outlier_train_features)
pca_test_features = pca.transform(scaled_no_outlier_test_features)


----
Tune model hyperparameters using validation data, could be cross-validation such as:
```
RandomizedSearchCV()
```

See **Model Training > Hyperparameter Tuning** Section


CV function description [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

---

In [ ]:
#import CV function

random_search = RandomizedSearchCV()
random_search.fit(pca_valid_features,valid_labels) #validation data
tuned_model_parameters = random_search.best_params_

---
Fit/Train ML models using the tuned model parameters and training data

See **Model Training > Fitting Linear Regression Model** Section


Information about different models [here](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning). 

---

In [ ]:
#import model function
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=150,max_depth=6)
rf_model.fit(pca_train_features,train_labels['OBS_sknt_max'].values) #training data


---
Evaluate model, change up dataset or training features if wanted

See **Model Evaluation** Section

Information about evalution metrics [here](https://scikit-learn.org/stable/modules/model_evaluation.html)

---

In [ ]:
# import metrics 

pred_labels = rf_model.predict(pca_test_features) #testing data

# Compare pred_labels to test_labels with metrics and/or reliability diagram




In [ ]:
# Reliability curve of high temperature linear regression model
regression_reliability_curve(pred_labels, test_labels['OBS_sknt_max'].values, nbins=20)

In [ ]:
from sklearn.metrics import mean_squared_error


wind_mse = mean_squared_error(pred_labels,test_labels['OBS_sknt_max'].values)

print(f'Mean square error: {wind_mse}')
print(f'\nPredicted values \n{pred_labels[:10]}')

print(f'\nActual values \n{test_labels["OBS_sknt_max"].values[:10] }')